# <center> <img src="../labs/img/ITESOLogo.png" alt="ITESO" width="480" height="130"> </center>
# <center> **Departamento de Electrónica, Sistemas e Informática** </center>
---
## <center> **Procesamiento de Datos Masivos** </center>
---
### <center> **Primavera 2025** </center>
---
### <center> **Consumer** </center>

---
**Profesor**: Dr. Pablo Camarillo Ramirez

In [1]:
import findspark
findspark.init()

#### Creacion de la conexión con el cluster de spark


In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("SparkSQLStructuredStreaming-Kafka") \
    .master("spark://80d04dce9402:7077") \
    .config("spark.ui.port","4040") \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.13:3.5.4") \
    .getOrCreate()
sc = spark.sparkContext

:: loading settings :: url = jar:file:/opt/conda/spark-3.5.4-bin-hadoop3-scala2.13/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.13 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-d72410d9-cabb-4cba-8cf1-0339de1e04cc;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.13;3.5.4 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.13;3.5.4 in central
	found org.apache.kafka#kafka-clients;3.4.1 in central
	found org.lz4#lz4-java;1.8.0 in central
	found org.xerial.snappy#snappy-java;1.1.10.5 in central
	found org.slf4j#slf4j-api;2.0.7 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.4 in central
	found org.apache.hadoop#hadoop-client-api;3.3.4 in central
	found commons-logging#commons-logging;1.1.3 in central
	found com.google.code.findbugs#jsr305;3.0.0 in central
	found org.scala-lang.modules#scala-parallel-collections_2.13;1.0.4 in central
	found org.apache.commons#commons-pool2;2.11.1 in centr

### Creación del Kafka Stream

In [3]:
from pyspark.sql.functions import from_json, col, current_timestamp
from pyspark.sql.types import StructType, FloatType, StringType

# Define schema matching the producer's data
schema = StructType() \
    .add("Close", FloatType()) \
    .add("High", FloatType()) \
    .add("Low", FloatType()) \
    .add("Open", FloatType()) \
    .add("Date", StringType()) \
    .add("Volume", FloatType())

# Read from Kafka
kafka_lines = spark \
    .readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "8e35f34f36db:9093") \
    .option("subscribe", "Kafka-train") \
    .load()



### Transform binary data into string

In [4]:
kafka_df = kafka_lines.withColumn("value_str", kafka_lines.value.cast("string")) \
    .withColumn("json_data", from_json(col("value_str"), schema)) \
    .select("json_data.*", current_timestamp().alias("timestamp"))

In [6]:

# Write to Parquet with simulated timestamps
query = kafka_df.writeStream \
    .format("parquet") \
    .option("path", "/home/jovyan/notebooks/data/streamdata/") \
    .option("checkpointLocation", "/home/jovyan/notebooks/data/checkpoint") \
    .outputMode("append") \
    .start()

query.awaitTermination()

25/05/11 19:53:08 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
25/05/11 19:53:08 WARN AdminClientConfig: These configurations '[key.deserializer, value.deserializer, enable.auto.commit, max.poll.records, auto.offset.reset]' were supplied but are not used yet.
ERROR:root:KeyboardInterrupt while sending command.                             
Traceback (most recent call last):
  File "/opt/conda/spark-3.5.4-bin-hadoop3-scala2.13/python/lib/py4j-0.10.9.7-src.zip/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/conda/spark-3.5.4-bin-hadoop3-scala2.13/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
                          ^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/conda/lib/python3.11/socket.py", line 706, in readinto
    

KeyboardInterrupt: 

In [ ]:
query.stop()

In [ ]:
# Path to the directory where Parquet files were written
parquet_path = "/home/jovyan/notebooks/data/streamdata/"

# Read the Parquet files into a DataFrame
parquet_df = spark.read.parquet(parquet_path)

# Show the first few rows of the DataFrame
parquet_df.show()

+---------+---------+---------+---------+----------+----------+--------------------+
|    Close|     High|      Low|     Open|      Date|    Volume|           timestamp|
+---------+---------+---------+---------+----------+----------+--------------------+
| 67.00368|  67.4405| 66.05723|  66.3994|2020-04-22|1.168624E8|2025-05-11 19:53:...|
| 66.74402|68.374825| 66.70519| 66.94787|2020-04-23|1.248144E8|2025-05-11 19:53:...|
| 68.71941| 69.05188| 67.93799| 68.38694|2020-04-27|1.170876E8|2025-05-11 19:53:...|
| 67.60553| 69.36495| 67.51332|69.182945|2020-04-28|1.120048E8|2025-05-11 19:53:...|
| 69.82604| 70.29684| 68.89416| 69.09801|2020-04-29|1.372808E8|2025-05-11 19:53:...|
| 70.15122| 72.56102|  69.3698|69.466866|2020-05-01|2.406168E8|2025-05-11 19:53:...|
| 71.14379| 71.27241|69.483864| 70.17551|2020-05-04| 1.33568E8|2025-05-11 19:53:...|
| 72.21156| 73.04638| 71.45926|71.604866|2020-05-05|1.477512E8|2025-05-11 19:53:...|
|  72.9566| 73.58999| 72.52948| 72.91534|2020-05-06|1.423336E8|20

In [ ]:
sc.stop()

In [17]:
from pyspark.sql.window import Window
from pyspark.sql.functions import col, lag, when

# Create a window spec to get previous day's close
window_spec = Window.orderBy("Date")

# Add previous day's close price
parquet_df = parquet_df.withColumn("prev_close", lag("Close").over(window_spec))

# Create binary label: 1 if price increased, else 0
classified_df = parquet_df.withColumn(
    "label", when(col("Close") > col("prev_close"), 1).otherwise(0)
).dropna()


In [23]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import RandomForestClassifier

feature_cols = ["Open", "High", "Low", "Volume"]
assembler = VectorAssembler(inputCols=feature_cols, outputCol="features")
data = assembler.transform(classified_df).select("features", "label")

# Train/test split
train, test = data.randomSplit([0.8, 0.2], seed=42)

# Train classifier
classifier = RandomForestClassifier(featuresCol="features", labelCol="label")
model = classifier.fit(train)

# Predictions
predictions = model.transform(test)
predictions.write.mode("overwrite").parquet("/home/jovyan/notebooks/data/predictions/")



25/05/11 20:26:47 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/05/11 20:26:47 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/05/11 20:26:47 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/05/11 20:26:49 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/05/11 20:26:49 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/05/11 20:26:49 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/05/11 2

In [22]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

evaluator_f1 = MulticlassClassificationEvaluator(metricName="f1")
evaluator_precision = MulticlassClassificationEvaluator(metricName="weightedPrecision")
evaluator_recall = MulticlassClassificationEvaluator(metricName="weightedRecall")

f1 = evaluator_f1.evaluate(predictions)
precision = evaluator_precision.evaluate(predictions)
recall = evaluator_recall.evaluate(predictions)

print(f"F1 Score: {f1}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")


25/05/11 20:25:09 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/05/11 20:25:09 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/05/11 20:25:09 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/05/11 20:25:12 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/05/11 20:25:12 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/05/11 20:25:12 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/05/11 2

F1 Score: 0.2857142857142857
Precision: 0.8809523809523809
Recall: 0.2857142857142857


25/05/11 20:25:17 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/05/11 20:25:17 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
